https://stackoverflow.com/questions/68003007/how-to-extract-text-boxes-from-a-pdf-and-convert-them-to-image
https://stackoverflow.com/questions/22898145/how-to-extract-text-and-text-coordinates-from-a-pdf-file


In [10]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
import pdfminer
import os
import pandas as pd
import pdf2image
import numpy as np
import PIL
from PIL import Image
import io
from pathlib import Path

from os import listdir
from os.path import isfile, isdir, join
import jieba


In [7]:
def pdfToImage(pdf_path):

    # PART 1: GET LTBOXES COORDINATES IN THE IMAGE
    # Open a PDF file.
    fp = open(pdf_path, 'rb')

    # Create a PDF parser object associated with the file object.
    parser = PDFParser(fp)

    # Create a PDF document object that stores the document structure.
    # Password for initialization as 2nd parameter
    document = PDFDocument(parser)

    # Check if the document allows text extraction. If not, abort.
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed

    # Create a PDF resource manager object that stores shared resources.
    rsrcmgr = PDFResourceManager()

    # Create a PDF device object.
    device = PDFDevice(rsrcmgr)

    # BEGIN LAYOUT ANALYSIS
    # Set parameters for analysis.
    laparams = LAParams()

    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)

    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)


    # here is where i stored the data
    boxes_data = []
    page_sizes = []

    
    def parse_obj(lt_objs, pageNum):
        # loop over the object list
        for obj in lt_objs:
            # if it's a textbox, print text and location
            if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
                #if verbose >0:
                #    print("%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.get_text()))
                #data_dict = {"startX":round(obj.bbox[0]),"startY":round(obj.bbox[1]),"endX":round(obj.bbox[2]),"endY":round(obj.bbox[3]),"text":obj.get_text()}
                data_dict = {"pageNum":pageNum, 
                             "startY":round(obj.bbox[1]),
                             "endY":round(obj.bbox[3]), 
                             "text":obj.get_text()}
                boxes_data.append(data_dict)
            # if it's a container, recurse
            elif isinstance(obj, pdfminer.layout.LTFigure):
                parse_obj(obj._objs, pageNum)


    # loop over all pages in the document
    for pageNum, page in enumerate(PDFPage.create_pages(document)):
        # read the page into a layout object
        interpreter.process_page(page)
        layout = device.get_result()
        # extract text from this object
        parse_obj(layout._objs, pageNum)
        mediabox = page.mediabox
        mediabox_data = {"height":mediabox[-1], "width":mediabox[-2]}
        page_sizes.append(mediabox_data)    
        
    return boxes_data
        



In [9]:

pdf_path = "/home/pancala/桌面/01中自1上課本pdf檔_111(1)/01中自1上編輯要旨課本_111(1).pdf.pdf"
boxes_data = pdfToImage(pdf_path)

for box in boxes_data:
    print(box["text"])


編輯要旨

一、本版教科書依據民國一○三年教育部發布之「十二年國民基本教育課程綱要總綱」及民國

一○七年發布之「十二年國民基本教育自然科學課程綱要」編輯而成。

二、本書於國中階段共分六冊，供自然科學領域教學，內容包含正式課程章節與書末延伸學習

資料，教師可視實際教學狀況，彈性調整教學架構。

三、教材編輯特色：

　　(一)圖、文編排精美活潑，融入美感教育。

　　(二)多元的學習方式。

　　(三)注重學習與生活的連結，及知識與概念的整合。

　　(四)探究實作與學習內容密切配合，扎實科學技能的運用。

　　(五)拓展生涯視野，培育正確的科學態度。

四、本書另編有活動紀錄簿，提供學生在實驗過程中做紀錄。

五、本書為協助教師教學，配合教學單元另編有備課用書。

六、本書所採用之圖照內容，已竭力取得授權，其來源出處刊登於書末。倘有疏漏，煩請著作

權人與本公司聯絡。

七、教材的完善，非一蹴可幾，教材的發展，更有賴使用者給予指導及建議，本教材如有不盡

妥適之處，敬請不吝指教，以供未來修訂之參考。

梅花鹿

p000_封面裡.indd   2
p000_封面裡.indd   2

2021/5/6   上午 10:20:42
2021/5/6   上午 10:20:42



In [16]:
book = []

# 指定要列出所有檔案的目錄
mypath = "/home/pancala/桌面/01中自1上課本pdf檔_111(1)/"

# 取得所有檔案與子目錄名稱
files = listdir(mypath)

# 以迴圈處理
for f in files:
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    # 判斷 fullpath 是檔案還是目錄
    if isfile(fullpath) and fullpath[-3:] == "pdf":
        print("檔案：", f)
        boxes_data = pdfToImage(fullpath)

        for box in boxes_data:
            #print(box["text"])
            book.append(box["text"])
print(book)



檔案： 02中自1上拉頁課本_111(1).pdf
檔案： 10中自1上第5章生物的恆定性課本_111(1).pdf
檔案： 12中自1上後共同頁課本_111(1).pdf
檔案： 06中自1上跨科主題-世界的各種大小樣貌課本_111(1).pdf
檔案： 07中自1上第2章養分課本_111(1).pdf
檔案： 04中自1上科學方法課本_111(1).pdf
檔案： 11中自1上科學工具箱課本_111(1).pdf
檔案： 09中自1上第4章生物的協調作用課本_111(1).pdf
檔案： 08中自1上第3章生物的運輸與防禦課本_111(1).pdf
檔案： 03中自1上目次課本_111(1).pdf
檔案： 05中自1上第1章生命的特性課本_111(1).pdf
檔案： 01中自1上編輯要旨課本_111(1).pdf.pdf
['本書使用說明\n22\n11\n', '  發現課題\n', '    學習與思考\n', '33\n', '觀察與探究\n', '44\n', '技能實作\n', '55\n', '整合知識\n', '目次 國 民 中 學\n', '第 一 冊\n', '立即掃瞄：\n動態圖表、扣課影\n音、評量遊戲，讓\n數位豐富學習。\n', '活學活用\n', '66\n', '章首與引言\n', '\x08進一步探索\n', '\x08觀念速記\n', '\x08\n', '\x08示範實驗\n', '科學工具箱\n', '學習地圖\n', '探究自然大小事\n', '各章開頭會有圖照與一段引\n言，引導你進入學習。\n', '腦力激盪或操作觀察後，\n和同學一起討論。\n', '幫 助 你 檢 查 是 否 理 解 剛\n', '剛學到的知識。\n', '專心看老師的示範，仔細思考\n相關問題。\n', '跟著科學家們，一起培養科學\n素養需要的技能。\n', '以圖表串連全章重點，帶你\n綜覽整章重要的學習內容。\n', '跨領域帶你認識更多有趣又實\n用的生活議題，一起來探究。\n', '緒論\n', '\x08自然暖身操\n', '\x08知識快遞\n', '逗逗 知識\n', '\x08探索活動\n', '概念連結\n', '科普閱讀\n', '科學方法\x08\n進入實驗室\x08\n', '2\n6\n', '一起來發現並思考生活

In [32]:
keywordSet = set()

allSentences = ""

for sentence in book:
    sentence = sentence.replace("\n","")
    sentence = sentence.replace("□","")
    sentence = sentence.replace(" ","")
    
    allSentences += sentence
    
seg_list = jieba.lcut(allSentences)
keywordSet.update(seg_list)
print(keywordSet)



{'】', '500mL', '0.7', '程序', '產出', '流包', '留下', '脈搏', '以氧氣', '江志緯', '避風', '1886', '成束', '因積', '舉出', '水水質', '外面', '38Q2Q2Q2', '由下而上', '準備', '體', '起設', '性及', '握住', '等待', '帶', '將小魚', '規模', '頭', '1.2', '大型犬', '184.285', '呃', '粉本氏', 'indd952021', 'j', '銜接成', '或經', '自大', '舞蹈', 'Q3Q3Q3', '冰水', '同學', '半小時', '剩餘', '情形', '多種', '鰾', '針種', '父親', '仍', '溼度', '組作', '碰觸後小葉', '107p086', '西林', '而能', '對待', '來臨', '他', '情感', '下節', '成品', '性腺', '身上', '焦調', '多肉', '對齊', '入侵', '留有', '介紹', '屬', '度越', '可同', '升糖', '126p126', '葉是', '金黃色', '各滴', '檢測劑', '116p086', '常常', '緊張', '其生長', '尊感覺', '具有', '關聯性', '疑疑', 'indd752021', '必要', '兩者', '鼠疫', '科學家將', '不自覺', '3474', '小腸', '時速', '胞膜', '運', '800', '？', '大後開花', '縫隙', '烘', '對甲試', '細胞洋', '類養', '有四顆', '脈', '歌', '直', '捕蟲', '離開', '犬階段', '321', 'indd144p126', 'indd177p158', 'indd182p158', '亮', '垂', '松鼠', '02', '103', '加熱', '絲', '腺肝', '突破', '009', '蝸牛殼', '物物', '利隊團', '免洗', '042021', '進蟻', 'indd712021', '一幕', '幅度', '痛而觸', '交會', '缺氧', '92A', '受器', '需施', '下襬', '次之', '碰', '否則', '最近', '確

In [35]:
len( keywordSet)

8825

In [33]:
allSentences

'本書使用說明2211發現課題學習與思考33觀察與探究44技能實作55整合知識目次國民中學第一冊立即掃瞄：動態圖表、扣課影音、評量遊戲，讓數位豐富學習。活學活用66章首與引言\x08進一步探索\x08觀念速記\x08\x08示範實驗科學工具箱學習地圖探究自然大小事各章開頭會有圖照與一段引言，引導你進入學習。腦力激盪或操作觀察後，和同學一起討論。幫助你檢查是否理解剛剛學到的知識。專心看老師的示範，仔細思考相關問題。跟著科學家們，一起培養科學素養需要的技能。以圖表串連全章重點，帶你綜覽整章重要的學習內容。跨領域帶你認識更多有趣又實用的生活議題，一起來探究。緒論\x08自然暖身操\x08知識快遞逗逗知識\x08探索活動概念連結科普閱讀科學方法\x08進入實驗室\x0826一起來發現並思考生活中有趣的科學問題吧，你可以在學習中找到答案！在旁支援的補充知識，讓你涉獵更廣。有趣的科普小補充，讓你成為知識王！自己動手做或與同學分工合作的小實驗。橫向統整跨章相關概念，幫助你學習融會貫通。\x08實驗實驗包含【放眼看自然】、【科學最前線】、【科學家故事】及【達人專欄】四大主題，帶你展望自然領域大未來。【觀察驗證】讓我們透過觀察實作驗證所學到的知識，了解自然現象的原理。【探究實證】讓我們運用科學方法，以實證精神來提出問題、形成假實驗2．1食物中醣類的測定實驗2．1實驗2．1【前言】食品包裝上營養標示的「碳水化合物」就是指醣類，我們日常吃的食物，例如米飯和水果都含有可產生能量的醣類。目的學習醣類的簡易測定方法，並藉由觀察碘液和本氏液的顏色變化，來推知食物中是否含有澱粉或葡萄糖等醣類。實驗器材（每組）白紙1張滴管3支酒精燈1盞量筒（10mL）1個碘液適量葡萄糖液（1％）適量標籤紙適量試管3支三腳架及陶瓷纖維網1組研缽及杵1組本氏液適量待測食物（熟米飯、芭樂和學生自行準備1種）適量載玻片3片試管夾1支燒杯（250mL）1個刮勺3支澱粉液（1％）適量碘液（又稱澱粉檢測劑）本氏液1.碘液是一種黃褐色試劑，可用來檢測食物1.本氏液是一種淡藍色試劑，可用來檢測食物中是否中是否含有澱粉。含有葡萄糖等醣類。2.將碘液加入食物後，如果顏色轉變成藍黑色或紫紅色，則表示食物中含有澱粉。2.如果食物中含有葡萄糖，將本氏液加入食物並加熱後，本氏液的顏色會改變（綠、黃、橙、紅），而且顏色越偏紅色，代表葡萄糖的濃度越

In [ ]:

        
    #抓取每一題的題號位置和D選項位置
    qNum = 1
    base = 0
    optionD = 0
    q_coordinate_list = []
    for box in boxes_data:
        if box["pageNum"] > 0 :
            if str(qNum)+"." in box["text"]:
                #print(box["text"])

                base += 1
                item = {}
                item["qNum"] = qNum            
                item["endY"] = box["endY"]

            if "(D)" in box["text"]:
                #print("------")
                #print(box["text"])
                optionD += 1
                item["startY"] = box["startY"]
                item["pageNum"] = box["pageNum"]
                q_coordinate_list.append(item)
                qNum += 1            
                    
                    
    #print(q_coordinate_list)
    # the magic numbers
    page_size = page_sizes[1]

    dpi = 200/72
    vertical_shift = 5 # I don't know, but it's need to shift a bit
    page_height = int(page_size["height"] * dpi)        
    
    
    
    path = Path().absolute()
    path3 = str(year) + "/" + subject + "/img題目/" 
    results_dir = os.path.join(path, path3 )

    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)

    #把題目圖片切下來
    # loop through boxes (we'll process only first page for now)
    for i, _ in enumerate(q_coordinate_list):

        #first box data
        startY = q_coordinate_list[i]["startY"]
        endY = q_coordinate_list[i]["endY"]
        pageNum = q_coordinate_list[i]["pageNum"]
        qNum = q_coordinate_list[i]["qNum"]


        # correction PDF --> PIL
        startY = page_height - int(startY * dpi) - vertical_shift
        endY   = page_height - int(endY   * dpi) - vertical_shift
        #startX = int(startX * dpi)
        #endX   = int(endX   * dpi)

        startX = int(65 * dpi)
        endX = int(540 * dpi)
        startY, endY = endY, startY 

        # turn image to array
        page_image = pdf2image.convert_from_path(pdf_path)[pageNum]

        image_array = np.array(page_image)
        # get cropped box
        box = image_array[startY:endY,startX:endX,:]
        convert2pil_image = PIL.Image.fromarray(box)
        #show cropped box image
        # convert2pil_image.show()


        png = path3 + str(qNum) + ".png"
        convert2pil_image.save(png)
        print(subject, qNum)


In [108]:
year = 111
subject_cName_list = ["國文", "英語", "數學", "自然", "社會"]
subject_eName_list = ["Chinese", "English", "Math", "Nature", "Society"]
'''
subject = "自然"
pdf_path ="歷屆會考題目pdf/108_Nature.pdf"

subject = "社會"
pdf_path ="歷屆會考題目pdf/108_Society.pdf"
'''

for i in range(5):
    subject = subject_cName_list[i]
    pdf_path ="歷屆會考題目pdf/" + str(year) + "P" + "_" + subject_eName_list[i] + ".pdf"
    pdfToImage(year, subject, pdf_path)



國文 1
國文 2
國文 3
國文 4
國文 5
國文 6
國文 7
國文 8
國文 9
國文 10
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
國文 11
英語 1
英語 2
英語 3
英語 4
英語 5
英語 6
英語 7
英語 8
英語 9
英語 10
英語 11
英語 12
英語 13
英語 14
英語 15
英語 16
英語 17
英語 18
英語 19
英語 20
英語 21
英語 22
英語 23
英語 24
英語 25
英語 26
英語 27
英語 28
英語 29
英語 30
英語 31
英語 32
英語 33
英語 34
英語 35
英語 36
英語 37
英語 38
英語 39
英語 40
英語 41
英語 42
英語 43
數學 1
數學 2
數學 3
數學 4
數學 5
數學 6
數學 7
數學 8
數學 9
數學 10
數學 11
數學 12
數學 13
數學 14
數學 15
數學 16
數學 17
數學 18
數學 19
數學 20
數學 21
數學 22
數學 23
數學 24
數學 25
自然 1
自然 2
自然 3
自然 4
自然 5
自然 6
自然 7
自然 8
自然 9
自然 10
自然 11
自然 12
自然 13
自然 14
自然 15
自然 16
自然 17
自然 18
自然 19
自然 20
自然 21
自然 22
自然 23
自然 24
自然 25
自然 26
自然 27
自然 28
自然 29
自然 30
自然 31
自然 32
自然 33
自然 34
自然 35
自然 36
自然 37
自然 38
自然 39
自然 40
自然 41
自然 42
自然 43
自然 44
自然 45
自然 45
自然 47
自然 48
自然 49
自然 50
社會 1
社會 2
社會 3
社會 4
社會 5
社會 6
社會 7
社會 8
社會 9
社會 10
社會 11
社會 12
社會 13
社會 14
社會 15
社會 16
社

# 以下測試

In [91]:
year = 108
'''
subject = "自然"
pdf_path ="歷屆會考題目pdf/108_Nature.pdf"

subject = "英語"
pdf_path ="歷屆會考題目pdf/108_English.pdf"

'''
subject = "社會"
pdf_path ="歷屆會考題目pdf/108_Society.pdf"

# PART 1: GET LTBOXES COORDINATES IN THE IMAGE
# Open a PDF file.
fp = open(pdf_path, 'rb')

# Create a PDF parser object associated with the file object.
parser = PDFParser(fp)

# Create a PDF document object that stores the document structure.
# Password for initialization as 2nd parameter
document = PDFDocument(parser)

# Check if the document allows text extraction. If not, abort.
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()

# Create a PDF device object.
device = PDFDevice(rsrcmgr)

# BEGIN LAYOUT ANALYSIS
# Set parameters for analysis.
laparams = LAParams()

# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)


# here is where i stored the data
boxes_data = []
page_sizes = []

def parse_obj(lt_objs, pageNum):
    # loop over the object list
    for obj in lt_objs:
        # if it's a textbox, print text and location
        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            #if verbose >0:
            #    print("%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.get_text()))
            #data_dict = {"startX":round(obj.bbox[0]),"startY":round(obj.bbox[1]),"endX":round(obj.bbox[2]),"endY":round(obj.bbox[3]),"text":obj.get_text()}
            data_dict = {"pageNum":pageNum, 
                         "startY":round(obj.bbox[1]),
                         "endY":round(obj.bbox[3]), 
                         "text":obj.get_text()}
            boxes_data.append(data_dict)
        # if it's a container, recurse
        elif isinstance(obj, pdfminer.layout.LTFigure):
            parse_obj(obj._objs, pageNum)

# loop over all pages in the document
for pageNum, page in enumerate(PDFPage.create_pages(document)):
    # read the page into a layout object
    interpreter.process_page(page)
    layout = device.get_result()
    # extract text from this object
    parse_obj(layout._objs, pageNum)
    mediabox = page.mediabox
    mediabox_data = {"height":mediabox[-1], "width":mediabox[-2]}
    page_sizes.append(mediabox_data)
    


In [92]:
boxes_data

[{'pageNum': 0, 'startY': 765, 'endY': 793, 'text': '請考生依指示\n填寫准考證末兩碼 \n'},
 {'pageNum': 0,
  'startY': 680,
  'endY': 754,
  'text': '108年國中教育會考\n社 會 科 試 題 本\n'},
 {'pageNum': 0, 'startY': 640, 'endY': 658, 'text': '請不要翻到次頁！\n'},
 {'pageNum': 0,
  'startY': 614,
  'endY': 632,
  'text': '讀完本頁的說明，聽從監試委員的指示才開始作答！\n'},
 {'pageNum': 0,
  'startY': 582,
  'endY': 595,
  'text': '※ 請先確認你的答案卡、准考證與座位號碼是否一致無誤。\n'},
 {'pageNum': 0, 'startY': 546, 'endY': 559, 'text': '請閱讀以下測驗作答說明：\n'},
 {'pageNum': 0, 'startY': 524, 'endY': 537, 'text': '測驗說明：\n'},
 {'pageNum': 0,
  'startY': 463,
  'endY': 521,
  'text': '這是國中教育會考社會科試題本，試題本採雙面印刷，共 14  頁，有 \n63 題選擇題，每題都只有一個正確或最佳的答案。測驗時間從  08：30 \n到 09：40，共 70 分鐘。作答開始與結束請聽從監試委員的指示。\n'},
 {'pageNum': 0, 'startY': 446, 'endY': 459, 'text': '注意事項：\n'},
 {'pageNum': 0,
  'startY': 406,
  'endY': 443,
  'text': '1. 所有試題均為四選一的選擇題，答錯不倒扣。\n2. 試題中所附圖形，如有附上比例尺，以比例尺為依據作答；若無 \n'},
 {'pageNum': 0, 'startY': 389, 'endY': 402, 'text': '比例尺，則該圖僅作為參考，不代表實際大小。\n'},
 {'pageNum': 0

In [93]:
#抓取每一題的題號位置和D選項位置
qNum = 1
base = 0
optionD = 0
q_coordinate_list = []
for box in boxes_data:
    if box["pageNum"] > 0 :
        if str(qNum)+"." in box["text"]:
            print(box["text"])

            base += 1
            item = {}
            item["qNum"] = qNum            
            item["endY"] = box["endY"]

        if "(D)" in box["text"]:
            print("------")
            print(box["text"])
            optionD += 1
            item["startY"] = box["startY"]
            item["pageNum"] = box["pageNum"]
            q_coordinate_list.append(item)
            qNum += 1            

q_coordinate_list

1. 

------
        (D)

2.  近年來我國政府審慎評估「農村引進農業外勞」的可行性，研擬該政策最主

------
(B)農業耕地狹小
(D)農產品商品化

3.  圖(一)是雜誌上的旅遊廣告，該行程的主要特色最

------
可能是下列何者？
(A)針葉林生態探索
(B)峽灣海岸郵輪行
(C)莽原動物大遷徙
(D)葡萄酒莊體驗遊

4.  「2016年初，滿載商品的貨運列車從中國 浙江 義烏出發，經西安、 蘭州和
烏魯木齊等城市後，駛往伊朗 德黑蘭，歷時14天完成首航，再現古絲路的輝
煌。」這班列車在駛離中國後，最可能途經下列何地？
(A)中亞 
(C)東歐 

------
(B)南亞
(D)東南亞

5.  圖(二)是阿政繪製某大洲海岸特色的成因分析， 
代表成因。該大洲最可能是下

------
(B)非洲
(D)歐洲

6.  臺灣因山林面積廣大、巡查不易，出現許多違法開發山林資源的事件。政府
機關若要快速比對不同時間的地表狀況，找出可能遭到大範圍違法伐林的地
點，使用下列哪一種地圖最為適合？
(A)分層設色圖 
(C)衛星影像圖 

------
(B)地形剖面圖
(D)等高線地形圖

7.  圖(三)分別是保存在新竹和
宜蘭的歷史文物，根據圖片
內容判斷，這些文物與下列
何者關係最密切？
(A)清朝人才選拔制度的施行
(B)鄭氏政權對於儒學的推廣
(C)臺灣總督府所實施的教育制度
(D)荷蘭 聯合東印度公司的文教政策

------
7.  圖(三)分別是保存在新竹和
宜蘭的歷史文物，根據圖片
內容判斷，這些文物與下列
何者關係最密切？
(A)清朝人才選拔制度的施行
(B)鄭氏政權對於儒學的推廣
(C)臺灣總督府所實施的教育制度
(D)荷蘭 聯合東印度公司的文教政策

8.  圖(四)是︽美國憲法 ︾某次修正案的內
容。此一修正案的通過最可能與下列哪
一事件有關？
(A)獨立宣言的頒布
(B)門羅宣言的發表
(C)南北戰爭的影響
(D)經濟大恐慌蔓延

------
8.  圖(四)是︽美國憲法 ︾某次修正案的內
容。此一修正案的通過最可能與下列哪
一事件有關？
(A)獨立宣言的頒布
(B)門羅宣言的發表
(C)南北戰爭的影響
(D)經濟大恐慌蔓延

9.  以下是對中國近代史上某組織的介紹：「他們號召建立一個土地財產公有和

[{'qNum': 1, 'endY': 740, 'startY': 669, 'pageNum': 1},
 {'qNum': 2, 'endY': 573, 'startY': 501, 'pageNum': 1},
 {'qNum': 3, 'endY': 489, 'startY': 382, 'pageNum': 1},
 {'qNum': 4, 'endY': 370, 'startY': 281, 'pageNum': 1},
 {'qNum': 5, 'endY': 256, 'startY': 166, 'pageNum': 1},
 {'qNum': 6, 'endY': 154, 'startY': 65, 'pageNum': 1},
 {'qNum': 7, 'endY': 768, 'startY': 625, 'pageNum': 2},
 {'qNum': 8, 'endY': 613, 'startY': 488, 'pageNum': 2},
 {'qNum': 9, 'endY': 476, 'startY': 368, 'pageNum': 2},
 {'qNum': 10, 'endY': 338, 'startY': 231, 'pageNum': 2},
 {'qNum': 11, 'endY': 201, 'startY': 75, 'pageNum': 2},
 {'qNum': 12, 'endY': 768, 'startY': 661, 'pageNum': 3},
 {'qNum': 13, 'endY': 649, 'startY': 542, 'pageNum': 3},
 {'qNum': 14, 'endY': 530, 'startY': 368, 'pageNum': 3},
 {'qNum': 15, 'endY': 356, 'startY': 231, 'pageNum': 3},
 {'qNum': 16, 'endY': 219, 'startY': 75, 'pageNum': 3},
 {'qNum': 17, 'endY': 768, 'startY': 517, 'pageNum': 4},
 {'qNum': 18, 'endY': 505, 'startY': 380, '

In [83]:
# the magic numbers
page_size = page_sizes[1]

dpi = 200/72
vertical_shift = 5 # I don't know, but it's need to shift a bit
page_height = int(page_size["height"] * dpi)

In [84]:
path = Path().absolute()
path3 = str(year) + "/" + subject + "/img題目/" 
results_dir = os.path.join(path, path3 )

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)


# loop through boxes (we'll process only first page for now)
for i, _ in enumerate(q_coordinate_list):

    #first box data
    startY = q_coordinate_list[i]["startY"]
    endY = q_coordinate_list[i]["endY"]
    pageNum = q_coordinate_list[i]["pageNum"]
    qNum = q_coordinate_list[i]["qNum"]
    

    # correction PDF --> PIL
    startY = page_height - int(startY * dpi) - vertical_shift
    endY   = page_height - int(endY   * dpi) - vertical_shift
    #startX = int(startX * dpi)
    #endX   = int(endX   * dpi)
    
    startX = int(65 * dpi)
    endX = int(540 * dpi)
    startY, endY = endY, startY 

    # turn image to array
    page_image = pdf2image.convert_from_path(pdf_path)[pageNum]

    image_array = np.array(page_image)
    # get cropped box
    box = image_array[startY:endY,startX:endX,:]
    convert2pil_image = PIL.Image.fromarray(box)
    #show cropped box image
    # convert2pil_image.show()
   
    
    png = path3 + str(qNum) + ".png"
    convert2pil_image.save(png)


    

In [ ]:
year = 108


subjectAndPath = {
    "自然":"歷屆會考題目pdf/108_Nature.pdf",
    "社會":"歷屆會考題目pdf/108_Society.pdf"
    "英語":"歷屆會考題目pdf/108_English.pdf"
    "國文":"歷屆會考題目pdf/108_Chinese.pdf"
    "數學":"歷屆會考題目pdf/108_Math.pdf"
}


for k,v in subjectAndPath.items():
    print(k,v)

## 測試

## Part 1

In [ ]:
# pdf path 
pdf_path ="test.pdf"

# PART 1: GET LTBOXES COORDINATES IN THE IMAGE
# Open a PDF file.
fp = open(pdf_path, 'rb')

# Create a PDF parser object associated with the file object.
parser = PDFParser(fp)

# Create a PDF document object that stores the document structure.
# Password for initialization as 2nd parameter
document = PDFDocument(parser)

# Check if the document allows text extraction. If not, abort.
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()

# Create a PDF device object.
device = PDFDevice(rsrcmgr)

# BEGIN LAYOUT ANALYSIS
# Set parameters for analysis.
laparams = LAParams()

# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)


# here is where i stored the data
boxes_data = []
page_sizes = []

def parse_obj(lt_objs, verbose = 0):
    # loop over the object list
    for obj in lt_objs:
        # if it's a textbox, print text and location
        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            if verbose >0:
                print("%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.get_text()))
            data_dict = {"startX":round(obj.bbox[0]),"startY":round(obj.bbox[1]),"endX":round(obj.bbox[2]),"endY":round(obj.bbox[3]),"text":obj.get_text()}
            boxes_data.append(data_dict)
        # if it's a container, recurse
        elif isinstance(obj, pdfminer.layout.LTFigure):
            parse_obj(obj._objs)

# loop over all pages in the document
for page in PDFPage.create_pages(document):
    # read the page into a layout object
    interpreter.process_page(page)
    layout = device.get_result()
    # extract text from this object
    parse_obj(layout._objs)
    mediabox = page.mediabox
    mediabox_data = {"height":mediabox[-1], "width":mediabox[-2]}
    page_sizes.append(mediabox_data)

## Part 2

In [ ]:
# loop over all pages in the document
for page in PDFPage.create_pages(document):
    # read the page into a layout object
    interpreter.process_page(page)
    layout = device.get_result()
    # extract text from this object
    parse_obj(layout._objs)
    mediabox = page.mediabox
    mediabox_data = {"height":mediabox[-1], "width":mediabox[-2]}
    page_sizes.append(mediabox_data)

# PART 2: NOW GET PAGE TO IMAGE -------------------------------------
firstpage_size = page_sizes[0]
firstpage_image = pdf2image.convert_from_path(pdf_path)[0] # without 'size=...'
#show first page with the right size (at least the one that pdfminer says)
# firstpage_image.show()
firstpage_image.save("firstpage.png")

# the magic numbers
dpi = 200/72
vertical_shift = 5 # I don't know, but it's need to shift a bit
page_height = int(firstpage_size["height"] * dpi)

# loop through boxes (we'll process only first page for now)
for i, _ in enumerate(boxes_data):

    #first box data
    startX, startY, endX, endY, text = boxes_data[i].values()

    # correction PDF --> PIL
    startY = page_height - int(startY * dpi) - vertical_shift
    endY   = page_height - int(endY   * dpi) - vertical_shift
    startX = int(startX * dpi)
    endX   = int(endX   * dpi)
    startY, endY = endY, startY 

    # turn image to array
    image_array = np.array(firstpage_image)
    # get cropped box
    box = image_array[startY:endY,startX:endX,:]
    convert2pil_image = PIL.Image.fromarray(box)
    #show cropped box image
    # convert2pil_image.show()
    png = "img/crop_" + str(i) + ".png"
    convert2pil_image.save(png)
    #print this does not match with the text, means there's an error
    print(text)

In [ ]:
pageNum

In [ ]:
max(5,3)

In [ ]:
idx = 1
q_base_coordinate_list = []
for box in boxes_data:
    if ".  " in box["text"]:
        #print(idx, box["pageNum"],box["startY"],box["endY"])
        q_base_coordinate_list.append([idx, box["pageNum"],box["startY"],box["endY"]])
        idx += 1
    if box["text"] == str(box["pageNum"]) + "\n":
        #print("*", box["pageNum"], box["startY"], box["endY"], box["text"])
        q_base_coordinate_list.append(["x", box["pageNum"],box["startY"],box["endY"]])
q_base_coordinate_list


In [ ]:
q_coordinate_list = []
for i in range(len(questionCoordinateList)):
    if q_base_coordinate_list[i][0] != 'x':
        q_end = q_base_coordinate_list[i][3]
        q_start = q_base_coordinate_list[i + 1][3]
        q_in_page = q_base_coordinate_list[i][1]
        q_coordinate_list.append([q_start, q_end, q_in_page])


q_coordinate_list


In [40]:
subjectAndPath = {
    "自然":"歷屆會考題目pdf/108_Nature.pdf",
    "社會":"歷屆會考題目pdf/108_Society.pdf",
    "英語":"歷屆會考題目pdf/108_English.pdf",
    "國文":"歷屆會考題目pdf/108_Chinese.pdf",
    "數學":"歷屆會考題目pdf/108_Math.pdf"
}



In [42]:
for k,v in subjectAndPath.items():
    print(k,v)

自然 歷屆會考題目pdf/108_Nature.pdf
社會 歷屆會考題目pdf/108_Society.pdf
英語 歷屆會考題目pdf/108_English.pdf
國文 歷屆會考題目pdf/108_Chinese.pdf
數學 歷屆會考題目pdf/108_Math.pdf


In [52]:
test = ".abac"
test[0:3]

'.ab'